[View in Colaboratory](https://colab.research.google.com/github/thayumaanavan/TensorFlow/blob/master/Intro_to_TensorFlow.ipynb)

In [0]:
from __future__ import absolute_import,division,print_function

import numpy as np
import tensorflow as tf

In [5]:
a = tf.constant(4.3)
b = tf.constant(2.4, dtype = tf.float32)
total = a + b
print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


In [0]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())